In [1]:
import util_las as las
import pandas as pd
import numpy as np
import pathlib

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Path for the previous and the new point cloud
prev_path = '/mnt/s3/proj-qalidar/02_Data/LiDAR_data/2018_NE_retiled/2546500_1212000.las'
new_path = '/mnt/s3/proj-qalidar/02_Data/LiDAR_data/2022_Neuchatel/lidar2022_classified/2546500_1212000.laz'
tile_name = '2546500_1212000' # Is used for the naming of the .csv outgoing file

classes_correspondance_path = '../classes_equivalence.csv' 
vox_xy = 1.5 # Voxel size in meters
vox_z = 1.5

In [3]:
prev_pc_df = las.las_to_df_xyzclass(prev_path)
new_pc_df = las.las_to_df_xyzclass(new_path)

# Remove all points which are noise in the previous generation as they do not bring useful information
prev_pc_df = prev_pc_df[prev_pc_df['classification']!=7]

# Match the supplementary class to classes from the previous generation
new_pc_df = las.reclassify(new_pc_df, classes_correspondance_path) 

# Set the lowest coordinates of the point clouds in each axis as the origin of the common grid 
x_origin = min(prev_pc_df.X.min(), new_pc_df.X.min())
y_origin = min(prev_pc_df.Y.min(), new_pc_df.Y.min())
z_origin = min(prev_pc_df.Z.min(), new_pc_df.Z.min())
# Same logic for the highest coordinates
x_max = max(prev_pc_df.X.max(), new_pc_df.X.max())
y_max = max(prev_pc_df.Y.max(), new_pc_df.Y.max())
z_max = max(prev_pc_df.Z.max(), new_pc_df.Z.max())

grid_origin = x_origin, y_origin, z_origin

grid_max = x_max, y_max, z_max

In [4]:
new_pc_df.head()

,X,Y,Z,classification
0,2546546.189,1212359.347,1001.970,2
1,2546546.077,1212359.614,1001.854,2
2,2546545.980,1212359.842,1001.831,2
3,2546545.889,1212360.058,1001.770,2
4,2546545.804,1212360.260,1001.689,2


In [5]:
prev_voxelised_df = las.to_voxelised_df(prev_pc_df, grid_origin, grid_max, vox_xy, vox_z)
new_voxelised_df = las.to_voxelised_df(new_pc_df, grid_origin, grid_max, vox_xy, vox_z)

In [6]:
def align_columns(df1, df2):
    # Modifiy the dataframes if one column is missing compared to the other. If it is the case it adds an empty column
    
    df1 = df1.copy(deep=True) # Do the modification on a copy of the dataframe
    df2 = df2.copy(deep=True)

    missing_columns_df1 = set(df2.columns) - set(df1.columns)

    for column in missing_columns_df1:
        df1[column] = pd.Series(dtype=df2[column].dtype)

    missing_columns_df2 = set(df1.columns) - set(df2.columns)

    for column in missing_columns_df2:
        df2[column] = pd.Series(dtype=df1[column].dtype)

    # Make sure that the order of the classification columns is sorted
    sorted_class_columns1 = df1.iloc[:,3:].reindex(sorted(df1.iloc[:,3:].columns), axis=1)
    df1.drop(df1.columns[3:], axis=1, inplace=True)
    df1 = pd.concat([df1, sorted_class_columns1],axis=1)

    sorted_class_columns2 = df2.iloc[:,3:].reindex(sorted(df2.iloc[:,3:].columns), axis=1)
    df2.drop(df2.columns[3:], axis=1, inplace=True)
    df2 = pd.concat([df2, sorted_class_columns2],axis=1)

    return df1, df2

In [7]:
# If one class is missing in either of the dataframe compared to the other, create new empty column
prev_voxelised_df, new_voxelised_df = align_columns(prev_voxelised_df, new_voxelised_df)

In [8]:
display(prev_voxelised_df.head(2))
display(new_voxelised_df.head(2))

,X_grid,Y_grid,Z_grid,1,2,3,6,7,17
0,2546500.75,1212000.75,1015.75,NaN,19.0,NaN,NaN,NaN,NaN
1,2546500.75,1212002.25,1015.75,NaN,15.0,NaN,NaN,NaN,NaN


,X_grid,Y_grid,Z_grid,1,2,3,6,7,17
0,2546500.75,1212000.75,1015.75,NaN,130.0,NaN,NaN,NaN,NaN
1,2546500.75,1212002.25,1015.75,NaN,122.0,NaN,NaN,NaN,NaN


In [9]:
# Free up space
del prev_pc_df
del new_pc_df

In [10]:
merged_df = prev_voxelised_df.merge(new_voxelised_df, on=['X_grid','Y_grid','Z_grid'], how='outer', suffixes=('_prev','_new'))

In [11]:
merged_df = merged_df.replace(np.NaN, 0)

In [12]:
merged_df.head()

,X_grid,Y_grid,Z_grid,1_prev,2_prev,3_prev,6_prev,7_prev,17_prev,1_new,2_new,3_new,6_new,7_new,17_new
0,2546500.75,1212000.75,1015.75,0.0,19.0,0.0,0.0,0.0,0.0,0.0,130.0,0.0,0.0,0.0,0.0
1,2546500.75,1212002.25,1015.75,0.0,15.0,0.0,0.0,0.0,0.0,0.0,122.0,0.0,0.0,0.0,0.0
2,2546500.75,1212003.75,1015.75,0.0,15.0,0.0,0.0,0.0,0.0,0.0,127.0,0.0,0.0,0.0,0.0
3,2546500.75,1212005.25,1015.75,0.0,17.0,0.0,0.0,0.0,0.0,0.0,126.0,0.0,0.0,0.0,0.0
4,2546500.75,1212006.75,1015.75,0.0,17.0,0.0,0.0,0.0,0.0,0.0,126.0,0.0,0.0,0.0,0.0


In [13]:
# Create the path for the folder to store the .csv file in case it doesn't yet exist
pathlib.Path('../out_dataframe/voxelised_comparison').mkdir(parents=True, exist_ok=True)

# In file name, set voxel size in centimeters, so as to avoid decimal (.) presence in the file name
merged_df.to_csv(f'../out_dataframe/voxelised_comparison/{tile_name}_{int(vox_xy*100)}-{int(vox_z*100)}'+'.csv', index=False)

## Sankey Diagram visualisation:
Note: the code is awful, not a priority, but might be good to go over it

In [117]:
newer_pc= las.las_to_df_xyzclass(new_path)

In [118]:
points_per_class = newer_pc.groupby('classification')['classification'].count().to_frame('nb_points').reset_index()

In [119]:
class_equivalences = pd.read_csv(classes_correspondance_path, sep=';')

In [135]:
matched_class_name = pd.DataFrame({'class_name':['Unclassified (1)','Ground (2)','Vegetation (3)','Buildings (6)','Noise (7)','Water (9)','Brigdes(17)'], 'id':[101,102,103,106,107,109,117]})

In [136]:
df_sancay = class_equivalences.merge(points_per_class, how='inner',left_on='id',right_on='classification')
df_sancay.matched_id=df_sancay.matched_id+100 # Simply to attribute other labels
df_sancay.head(10)

,id,class_name,matched_id,classification,nb_points
0,1,Unclassified,101,1,117150
1,2,Ground,102,2,6030981
2,3,Low vegetation,103,3,367313
3,4,Medium vegetation,103,4,910833
4,5,High vegetation,103,5,6036875
5,6,Building roofs,106,6,2317897
6,7,Low Point (Noise),107,7,10849
7,11,"Piles, heaps (natural materials)",101,11,12006
8,15,"Masts, antenas",101,15,7477
9,17,Bridges,117,17,1079


In [137]:
all_nodes = pd.concat([df_sancay[['id','class_name']],matched_class_name]).reset_index(drop=True)
all_nodes.head()

,id,class_name
0,1,Unclassified
1,2,Ground
2,3,Low vegetation
3,4,Medium vegetation
4,5,High vegetation


In [138]:
df_sancay.merge(all_nodes.reset_index(),left_on='matched_id', right_on='id', how='inner').head()

,id_x,class_name_x,matched_id,classification,nb_points,index,id_y,class_name_y
0,1,Unclassified,101,1,117150,18,101,Unclassified (1)
1,11,"Piles, heaps (natural materials)",101,11,12006,18,101,Unclassified (1)
2,15,"Masts, antenas",101,15,7477,18,101,Unclassified (1)
3,19,Street lights,101,19,4269,18,101,Unclassified (1)
4,21,Cars,101,21,57585,18,101,Unclassified (1)


In [139]:
df_sancay.merge(all_nodes.reset_index(),left_on='matched_id', right_on='id', how='inner').sort_values(by='id_x')['index']

0     18
7     19
9     20
10    20
11    20
12    21
15    22
1     18
2     18
17    24
16    22
3     18
4     18
13    21
5     18
14    21
6     18
8     19
Name: index, dtype: int64

In [140]:
df_sancay

,id,class_name,matched_id,classification,nb_points
0,1,Unclassified,101,1,117150
1,2,Ground,102,2,6030981
2,3,Low vegetation,103,3,367313
3,4,Medium vegetation,103,4,910833
4,5,High vegetation,103,5,6036875
5,6,Building roofs,106,6,2317897
6,7,Low Point (Noise),107,7,10849
7,11,"Piles, heaps (natural materials)",101,11,12006
8,15,"Masts, antenas",101,15,7477
9,17,Bridges,117,17,1079


In [134]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = all_nodes.class_name,#["A1", "A2", "B1", "B2", "C1", "C2"],
      color = "blue"
    ),
    link = dict(
      source = np.arange(0,18), # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = df_sancay.merge(all_nodes.reset_index(),left_on='matched_id', right_on='id', how='inner').sort_values(by='id_x')['index'],
      value = df_sancay.nb_points
  ))])

fig.update_layout(
    autosize=False,
    width=800,
    height=800,
)

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()